<a href="https://colab.research.google.com/github/satyamverma95/DL_NLP/blob/main/Code_Explanation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [ ]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )


- The function preprocess_function takes a dictionary examples as input.</br>
- It extracts the "question" field from examples and stores each question in a list called questions. The strip() method is used to remove any leading or trailing whitespace from each question.</br>
- The tokenizer is then called with the following parameters:</br>
- The questions list as the first argument.</br>
- The "context" field from examples as the second argument.</br>
- max_length=384 sets the maximum length of the tokenized inputs to 384 tokens.
truncation="only_second" truncates the context if it exceeds the maximum length, keeping the question intact.</br>
- return_offsets_mapping=True instructs the tokenizer to return the mapping between the tokens and the original character offsets.</br>
- padding="max_length" pads the inputs to the maximum length.

1) max_length=384 sets the maximum length of the tokenized inputs to 384 tokens.
Explain this with example

Ans) Certainly! The max_length=384 parameter in the code sets the maximum length of the tokenized inputs to 384 tokens. This means that after tokenization, the resulting input sequence will be truncated or padded to a maximum length of 384 tokens.

Let's consider an example to understand this better:

Context: "The quick brown fox jumps over the lazy dog."
Question: "What color is the fox?"

To tokenize this example, we'll use a hypothetical tokenizer that splits the input into individual words:

Tokenized context: ["The", "quick", "brown", "fox", "jumps", "over", "the", "lazy", "dog", "."]
Tokenized question: ["What", "color", "is", "the", "fox", "?"]

Now, let's see how the max_length=384 parameter affects the tokenized inputs.

In this example, both the context and the question have very few tokens, so the resulting input sequence length will be well below the maximum length of 384 tokens. Therefore, no truncation or padding is required.

However, if we consider a longer context and question, such as:

Context: "Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua."
Question: "What is the meaning of Lorem ipsum?"

After tokenization, the input sequences would look like this:

Tokenized context: ["Lorem", "ipsum", "dolor", "sit", "amet", ",", "consectetur", "adipiscing", "elit", ",", "sed", "do", "eiusmod", "tempor", "incididunt", "ut", "labore", "et", "dolore", "magna", "aliqua", "."]
Tokenized question: ["What", "is", "the", "meaning", "of", "Lorem", "ipsum", "?"]

In this case, the total number of tokens in the context and question combined exceeds 384 tokens. Therefore, truncation or padding will be applied to ensure that the total length does not exceed the maximum length.

Truncation: If the total length exceeds 384 tokens, the tokenizer will truncate the input sequence by removing tokens from the end of the sequence until it reaches the maximum length.

Padding: If the total length is less than 384 tokens, the tokenizer will add special padding tokens (e.g., <pad>) to the input sequence until it reaches the maximum length.

By setting max_length=384, the code ensures that the tokenized inputs are limited to a maximum length of 384 tokens, allowing for consistent input size during model training or evaluation. 

2) return_offsets_mapping=True, Explain this with example?
Ans) Certainly! The return_offsets_mapping=True parameter in the code instructs the tokenizer to return the mapping between the tokens and the original character offsets in the input text. This mapping provides the information about which characters in the original text correspond to each token in the tokenized sequence.

Let's consider an example to understand this better:

Context: "The quick brown fox jumps over the lazy dog."

Tokenized context: ["The", "quick", "brown", "fox", "jumps", "over", "the", "lazy", "dog", "."]

When return_offsets_mapping=True, the tokenizer will also return the offsets mapping, which is a list of tuples representing the start and end character offsets for each token.

Offsets mapping: [(0, 3), (4, 9), (10, 15), (16, 19), (20, 25), (26, 30), (31, 34), (35, 39), (40, 43), (43, 44)]

Each tuple in the offsets mapping represents the start and end character offsets for a token. For example, the first tuple (0, 3) corresponds to the token "The", which starts at character index 0 and ends at character index 3 in the original text.

This offsets mapping provides a way to align the tokens back to their original positions in the input text. It can be useful for tasks like extracting answers from a given context or highlighting specific tokens in the original text.

In the context of the provided code, the offsets mapping is used to determine the character offsets of the answer span in the original text and label the corresponding start and end positions of the answer span in the tokenized sequence.

Overall, by setting return_offsets_mapping=True, the code enables the retrieval of the offsets mapping, allowing for better alignment between tokens and the original text, which can be helpful in various downstream tasks.

3) truncation="only_second", Explain with example?
Ans) The `truncation="only_second"` parameter in the code specifies the truncation strategy to be applied when tokenizing the inputs. In this case, it means that if the combined length of the question and the context exceeds the maximum length allowed (`max_length=384`), only the context will be truncated, and the question will be kept intact.

Let's consider an example to understand this better:

Context: "The quick brown fox jumps over the lazy dog."
Question: "What color is the fox?"

Tokenized context: ["The", "quick", "brown", "fox", "jumps", "over", "the", "lazy", "dog", "."]
Tokenized question: ["What", "color", "is", "the", "fox", "?"]

In this example, the combined length of the tokenized question and the tokenized context is 11. If the maximum length is set to 10, truncation will be necessary to fit the inputs within the specified maximum length.

With `truncation="only_second"`, only the context will be truncated, and the question will be kept intact. The resulting tokenized inputs will look like this:

Truncated tokenized context: ["The", "quick", "brown", "fox", "jumps", "over", "the", "lazy", "dog"]
Tokenized question: ["What", "color", "is", "the", "fox", "?"]

As you can see, the context has been truncated by removing the last token ".", while the question remains unchanged.

By using `truncation="only_second"`, the code ensures that the question is preserved as much as possible, even if it means truncating the context. This can be useful in question answering tasks, where the question plays a crucial role in understanding the context and generating accurate answers.

In [ ]:
    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []


The "offset_mapping" is extracted from the inputs dictionary using the pop() method and assigned to the offset_mapping variable. This mapping represents the correspondence between the tokens and the original character offsets in the context.
The "answers" field from examples is assigned to the answers variable.
Empty lists, start_positions and end_positions, are created to store the start and end positions of the answer.